In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow import keras
import time

C:\Users\Tsvi\AppData\Local\Programs\Python\Python36\lib\site-packages\OpenSSL\_util.py:6: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.
  from cryptography.hazmat.bindings.openssl.binding import Binding


In [ ]:
def Send_to_API(base_name,dataset_name, top_layer_inputs, labels):
        try:
            X_j = top_layer_inputs.tolist() # inputs
            labels_j = labels.tolist() #labels
        
        username = os.getenv('JUPYTERHUB_USER')
        payload = {'username' : username, 'base': base_name, 'data' : dataset_name, 'X': X_j, 'y': labels_j}
        headers = { 'content-type': 'application/json', 'accept': 'application/json' }
        
        r = requests.post(url = 'https://om-learn-api.azurewebsites.net/api/train', data = json.dumps(payload), headers=headers)
        result = json.loads(r.text)
        
        return(result)
        
    except Exception as e:
        return(e.msg)

In [2]:
from datetime import datetime
now=datetime.now()
date_time=now.strftime("%m-%d-%Y")

In [3]:
date_time

'05-09-2023'

In [4]:
print(tf.__version__)

2.6.2


In [5]:
BATCH_SIZE =1 #32   #1
IMG_SIZE = (160, 160)

In [6]:
# I left this because it sets up the directories automatically  
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)

In [7]:
# Different datasets to benchmark
#dataset_name='Animals_filtered'  # 50 animal dataset
dataset_name='cats_and_dogs_filtered'  #originalTF notebook dataset "full"
#dataset_name='fowl_data' # from https://learn.microsoft.com/en-us/azure/machine-learning/how-to-train-pytorch

PATH2 = os.path.join(os.path.dirname(path_to_zip),dataset_name )    
# also put in original
print(PATH2)

C:\Users\Tsvi\.keras\datasets\cats_and_dogs_filtered


In [8]:
train_dir = os.path.join(PATH2, 'train')
validation_dir = os.path.join(PATH2, 'validation')

#print(third_animal_dataset_path,'\n',third_animal_train_dir,'\n',third_animal_validation_dir)
print(PATH2,'\n',train_dir,'\n',validation_dir)

C:\Users\Tsvi\.keras\datasets\cats_and_dogs_filtered 
 C:\Users\Tsvi\.keras\datasets\cats_and_dogs_filtered\train 
 C:\Users\Tsvi\.keras\datasets\cats_and_dogs_filtered\validation


In [9]:
train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                                         shuffle=True,
                                                                         batch_size=BATCH_SIZE,
                                                                         image_size=IMG_SIZE)

validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                                              shuffle=True,
                                                                              batch_size=BATCH_SIZE,
                                                                              image_size=IMG_SIZE)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.


In [10]:
#print("original dataset class names:", train_dataset.class_names)
print("Dataset class names:", train_dataset.class_names)

Dataset class names: ['cats', 'dogs']


In [11]:
class_names = train_dataset.class_names
num_outputs=len(class_names)
print('number of classes and outputs:',num_outputs)

number of classes and outputs: 2


In [12]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

In [13]:
print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

Number of validation batches: 800
Number of test batches: 200


In [14]:
print('Number of training batches: %d' % tf.data.experimental.cardinality(train_dataset))

Number of training batches: 2000


In [15]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [16]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [17]:
#preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [18]:
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

In [19]:
#Training the model 10 epochs
initial_epochs = 1 #10  

In [20]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
#base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

In [21]:
MODEL = 'MobileNetV2'
#MODEL = 'ResNet50'


In [22]:
if MODEL=='MobileNetV2':
    preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
    base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                                include_top=False,
                                                weights='imagenet')
elif MODEL=='ResNet50':
    preprocess_input = tf.keras.applications.resnet50.preprocess_input
    base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,
                                                include_top=False,
                                                weights='imagenet')
else:
    error("Model Not supported")


In [23]:
base_model.trainable = False

In [24]:
# Let's take a look at the base model architecture
base_model.summary()

Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 80, 80, 32)   0           bn_Conv1[0][0]                   
_______________________________________________________________________________

In [25]:
# the original adds 2d global_average_pooling2d layer to make the base layer ready for classification.  
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [26]:
# now I set up traditional learning and top layer, but I change from binary to multiclass so I must add softmax to top
final_output_layer=tf.keras.layers.Softmax()

In [27]:
# This is the multiclass equivalent to original transfer learning network  
prediction_layer4 = tf.keras.layers.Dense(num_outputs)  # change1 from original: I make num_outputs outputs
inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = prediction_layer4(x)    # adds the 49 nodes
outputs = final_output_layer(x)  # change2: adds softmax
orig_TF_paradigm_model = tf.keras.Model(inputs, outputs)   # this is now the traditional leanring network

In [28]:
# Change 3:SparseCategoricalCrossentropy from BinaryCrossentropy because it is multiclass

base_learning_rate = 0.0001
orig_TF_paradigm_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  #loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'],
              run_eagerly=True,) # Now I compile it.  

In [29]:
orig_TF_paradigm_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 160, 160, 3)       0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 160, 160, 3)       0         
_________________________________________________________________
tf.math.subtract (TFOpLambda (None, 160, 160, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_160 (Functi (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0     

In [30]:
# here I create the version of the base model that will feed inputs to OM layer and I call it model_with_av_layer
# it does not have anything after the global_average_layer
inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
outputs = global_average_layer(x)
#outputs = prediction_layer(x)   # It is basically the same as the original notebook but no prediction layer
model_with_av_layer = tf.keras.Model(inputs, outputs) 


In [31]:
model_with_av_layer.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 160, 160, 3)       0         
_________________________________________________________________
tf.math.truediv_1 (TFOpLambd (None, 160, 160, 3)       0         
_________________________________________________________________
tf.math.subtract_1 (TFOpLamb (None, 160, 160, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_160 (Functi (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
Total params: 2,257,984
Trainable params: 0
Non-trainable params: 2,257,984
_________________________________________________

In [32]:
# Set up OM model that will take inputs from the outputs of model_with_av_layer

In [33]:
from OM_Model_API import OM_CORE

In [34]:
num_inputs=model_with_av_layer.output.shape[1]  #1280

OMmodel = OM_CORE(num_inputs, num_outputs)

Version: OM core 0.3 5/9/2023


In [35]:
#in case it is run or tested un-initiated  
OMmodel.compile()

In [36]:
# this is an equivalent TF model that will recieve the OM weights
prediction_layer2 = tf.keras.layers.Dense(num_outputs)  # top nodes
inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = prediction_layer2(x)  # nodes whose weights I will be changed by om
outputs = final_output_layer(x)  #softmax
tf_model_tobe_trained_by_OM = tf.keras.Model(inputs, outputs)


In [37]:
#it needs compile if I do validation but I never really do any learning so most of this doesnt really matter
base_learning_rate = 0.0001
tf_model_tobe_trained_by_OM.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  #loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'],
              run_eagerly=True,) # Efi: needs to be added to prevent execution optimization (not data optimiziation)

In [38]:
tf_model_tobe_trained_by_OM.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 160, 160, 3)       0         
_________________________________________________________________
tf.math.truediv_2 (TFOpLambd (None, 160, 160, 3)       0         
_________________________________________________________________
tf.math.subtract_2 (TFOpLamb (None, 160, 160, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_160 (Functi (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0   

In [39]:
# so I dont validate at every point for graphing (otherwise things take forever)
validate_points=np.concatenate([np.arange(30),np.arange(30,40,2),np.arange(40,60,3),np.arange(60,100,5),np.arange(100,300,7),np.arange(300,700,10),np.arange(700,1500,15),np.arange(1500,3000,20)])

In [40]:
len(validate_points)

248

In [41]:
# start re-run here 

In [42]:
# find the number of entries in thrid_animal_validation set: there doesnt seem to be an adequate internal function to do this!
# Im sure there is a better way but whatever
num_data_in_validation_dataset=0
for data, label in validation_dataset: #.as_numpy_iterator():
    num_data_in_validation_dataset=num_data_in_validation_dataset+len(data)
print(num_data_in_validation_dataset)

800


In [43]:
# setup validation manually so I dont have to go through base for every validation (saves time because validation is the same)
i=0
validation_labels=np.ones((1,num_data_in_validation_dataset))*50  #crazy initial values for debug
net_out_validation_for_OM=np.ones((num_data_in_validation_dataset,num_inputs))*50  

for data, label in validation_dataset: #.as_numpy_iterator():
    #print("Batch",i,"out of",num_data_in_validation_dataset,"size:",len(data),len(label))
    len_batch=len(label)
    #print(label)
    validation_labels[0,(BATCH_SIZE*i):(BATCH_SIZE*(i)+len_batch)]=label  # set up validation truth table
    temp=model_with_av_layer.predict(data)  #run bottom transfer layers pre-rfn
    net_out_validation_for_OM[BATCH_SIZE*i:(BATCH_SIZE*i+len_batch),:]=temp[0:len_batch,:]
    i=i+1
validation_labels=validation_labels.astype(int)


In [44]:
print(validation_labels[0,:])

[0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 0 1 1 0 0 0 0 1 0 1 1 1 0 0 1 1 1 1 1 1 0 1
 1 1 1 0 0 1 1 0 1 0 0 0 0 1 0 1 0 1 0 1 1 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0
 1 0 1 1 0 1 1 0 0 1 0 1 0 0 1 1 1 0 1 1 0 0 0 0 0 0 1 1 0 1 0 1 1 0 1 1 0
 1 1 0 1 0 0 0 0 0 0 1 0 0 1 0 1 1 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 1 0 1 1 0
 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 0 1 1 1 0 0 0 1 0 1 1 0 0 1 1 0 1 1 0 1 0 1
 1 0 1 1 0 0 0 0 0 1 1 0 1 1 1 0 1 1 1 1 0 0 0 1 1 0 0 0 1 0 1 1 1 1 0 1 0
 1 0 1 1 1 0 1 0 0 1 0 1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 0 0 1 1 1 1 0 1 1 0 1
 0 0 1 0 1 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 1 0 1 1 0 1 0 1 0 0 1 0 1 0 1 1
 1 1 0 0 1 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0 1 1 0 0 0 0 0 1 0 1 1 0 1 1 0 1 1
 0 1 1 0 1 1 1 1 0 0 0 1 1 1 0 1 1 0 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 1 1
 0 0 0 0 1 1 1 0 1 1 1 1 1 0 0 1 1 0 1 0 0 1 0 1 0 1 0 1 1 0 1 0 0 1 1 0 0
 0 0 0 1 0 1 1 1 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 1 1 0 1 1 0 1 1 0 1 0 1 0 0
 0 1 1 0 1 0 1 0 0 0 1 0 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 0 0 0 0 1 0 1 0 1 1
 0 0 0 1 0 0 0 0 0 0 0 0 

In [45]:
# for batches
num_batches_in_train_dataset=len(train_dataset)              # number of batches in train_dataset
print("num of data of train batches in a single epoch",num_batches_in_train_dataset,"batch size =",BATCH_SIZE,"epochs=",initial_epochs)
OM_learn_Vacc=[] #0.5*np.ones(num_batches_in_train_dataset*initial_epochs)    #initializing as .5 just in case zeros gets through somehow  #np.zeros(num_batches_in_train_dataset)
tf_learn_Vacc=[] #0.5*np.ones(num_batches_in_train_dataset*initial_epochs)    #np.zeros(num_batches_in_train_dataset)

num of data of train batches in a single epoch 2000 batch size = 1 epochs= 1


In [46]:
labels_presented=[]

In [47]:
tf_learn_Vacc

[]

In [48]:
# store baselines before learn
store_baselines_before_learning=False
if store_baselines_before_learning:
    loss_, accuracy_ = orig_TF_paradigm_model.evaluate(validation_dataset)
    print('validation of TF before train on new',accuracy_)
    tf_learn_Vacc.append(accuracy_) #store data

    OMmodel.test_data_score(net_out_validation_for_OM,validation_labels[0,:])  #validate for OM
    print('validation of OM on NEW data before train on new',OMmodel.last_score_accuracy/100)
    OM_learn_Vacc.append(OMmodel.last_score_accuracy/100) 
    
    # I want to add one more zero point and increase the index of everything by 1 to test before learning
    graph_points=list(validate_points.copy()+1)
    graph_points.insert(0,0)
else:
    graph_points=list(validate_points.copy())


In [49]:
initial_epochs=1

In [ ]:
for j in range(initial_epochs):
    if j>0:  #update graph points if doing more than one epoch
        validate_points=np.concatenate([np.arange(0,3000,30)])
        graph_points[num_stored:]=[]
        graph_points=graph_points+list(validate_points.copy()+len(labels_presented)+1)   #need to figure out graphing stuff

    for i, data in train_dataset.enumerate():  #seems similar to fit 
        
        labels_presented.append(data[1])  # store record of labels presented
        
        #t = time.time()
        
        #the original learning        
        batch_logs = orig_TF_paradigm_model.train_step(data)
        
        #TF_time=time.time() - t

        
        #t = time.time()      # I include time to percolate data through base.  
        
        # prepare data to send out to OM.        
        net_out = model_with_av_layer.predict_on_batch(data[0])  # run through base layer to have data ready for OM
        
        
        OMmodel.learn(net_out,data[1])  # needs data post base and label

        # call to the API
        
        Send_to_API(MODEL,dataset_name, top_layer_inputs, labels)
        
        json_back=Get_from_API(MODEL,dataset_name)
        
        u_model=json_back['model']
        
        
        
        
        
        
        #OM_time=time.time() - t
        
        #print('Learning Time  OM:',np.round(OM_time,4),'  TF:',np.round(TF_time,4),'  TF/OM=',np.round(TF_time/OM_time,4),'X')
        
        # this would be equivalent to a callback from the API getting back the weights from the API
        # the TF structure recieves the weights that come back put into the trainable layer: OMmodel.u_model
        tf_model_tobe_trained_by_OM.trainable_weights[0].assign(tf.Variable(np.float32(OMmodel.u_model.T))) 
        # this is now the TF equivalent of OM learned net 
        
        #this is code to validate but it is repeated in the validated_points
        #loss0, accuracy0 = tf_model_tobe_trained_by_OM.evaluate(validation_dataset)  # it can be validated and run just like the original
        
        
        if i in validate_points:  # minimizing number of validations because it takes too long
        
            # validating TF learned top layer
            loss_, accuracy_ = orig_TF_paradigm_model.evaluate(validation_dataset)

            # validating directly with OM, This function is not needed in the API, just the tf_model_tobe_trained_by_OM code (a few down)
            OMmodel.test_data_score(net_out_validation_for_OM,validation_labels[0,:])  #,verbose=True) 
                
            # adding TF accuracies to the record
            tf_learn_Vacc.append(accuracy_)  
            
            # adding OM  accuracies to the records
            OM_learn_Vacc.append(OMmodel.last_score_accuracy/100) # using direct validation

            if False:  # extra sanity validation to show OMTF == OM core but extra tests take too much time if you are training the net
            
                # validating OM learned top layer but equivalent net to TF
                loss0, accuracy_OMTF = tf_model_tobe_trained_by_OM.evaluate(validation_dataset)  # it is validated and run just like the original TF
                #OM_learn_Vacc.append(accuracy_OMTF)

                print("Training: Batch {}, Epoch {}, {} entries accuracy of OM core {} OMTF {} (should be roughly equivalent to OM) and TF {} ".format(i,j, len(data[0]), OM_learn_Vacc[-1],accuracy_OMTF,accuracy_))
            
            else:
                print("Training: Batch {}, Epoch {}, {} entries accuracy of OM core {} and TF {} ".format(i,j, len(data[0]), np.round(OM_learn_Vacc[-1],4),np.round(accuracy_,4)))

        
    
    num_stored=len(OM_learn_Vacc)
    
    if False:  # epoch plot not implemented in this version
        OM_epoch_Vacc_old[j]=sum(OM_learn_Vacc_old[(j*num_batches_in_train_dataset):((j+1)*num_batches_in_train_dataset)])/num_batches_in_train_dataset
        tf_epoch_Vacc_old[j]=sum(tf_learn_Vacc_old[(j*num_batches_in_train_dataset):((j+1)*num_batches_in_train_dataset)])/num_batches_in_train_dataset
        print("Epoch {} summing {} batches to get epoch accuracy of OM on new {} old {} and TF on new {} old {} data".format(j,i,OM_epoch_Vacc[j],OM_epoch_Vacc_old[j],tf_learn_Vacc[j],tf_epoch_Vacc_old[j]))



 Learned [1] of label(s) [1] , previously [0. 0.] of each.
800/800 [==============================] - 51s 63ms/step - loss: 0.7711 - accuracy: 0.41251s - loss: 0.7710 - accuracy:  - ETA: 
 (Number correct, number incorrect, number of tests): ( 391 409 800 ) %Correct:  48.875 %
False Positives by label: [  0. 409.]
False Negatives by label: [409.   0.]
See   .last_score_accuracy for % accuracy value
Training: Batch 0, Epoch 0, 1 entries accuracy of OM core 0.4888 and TF 0.4125 
 Learned [1] of label(s) [1] , previously [0. 1.] of each.
800/800 [==============================] - 54s 67ms/step - loss: 0.7724 - accuracy: 0.4175
 (Number correct, number incorrect, number of tests): ( 391 409 800 ) %Correct:  48.875 %
False Positives by label: [  0. 409.]
False Negatives by label: [409.   0.]
See   .last_score_accuracy for % accuracy value
Training: Batch 1, Epoch 0, 1 entries accuracy of OM core 0.4888 and TF 0.4175 
 Learned [1] of label(s) [1] , previously [0. 2.] of each.
800/800 [======

 Learned [1] of label(s) [0] , previously [19. 22.] of each.
 Learned [1] of label(s) [0] , previously [20. 22.] of each.
 Learned [1] of label(s) [1] , previously [21. 22.] of each.
800/800 [==============================] - 46s 57ms/step - loss: 0.7248 - accuracy: 0.4825
 (Number correct, number incorrect, number of tests): ( 773 27 800 ) %Correct:  96.625 %
False Positives by label: [ 9. 18.]
False Negatives by label: [18.  9.]
See   .last_score_accuracy for % accuracy value
Training: Batch 43, Epoch 0, 1 entries accuracy of OM core 0.9662 and TF 0.4825 
 Learned [1] of label(s) [0] , previously [21. 23.] of each.
 Learned [1] of label(s) [0] , previously [22. 23.] of each.
 Learned [1] of label(s) [0] , previously [23. 23.] of each.
800/800 [==============================] - 46s 57ms/step - loss: 0.7209 - accuracy: 0.4837
 (Number correct, number incorrect, number of tests): ( 772 28 800 ) %Correct:  96.5 %
False Positives by label: [10. 18.]
False Negatives by label: [18. 10.]
See

 Learned [1] of label(s) [0] , previously [46. 49.] of each.
800/800 [==============================] - 46s 57ms/step - loss: 0.6681 - accuracy: 0.5775
 (Number correct, number incorrect, number of tests): ( 773 27 800 ) %Correct:  96.625 %
False Positives by label: [ 9. 18.]
False Negatives by label: [18.  9.]
See   .last_score_accuracy for % accuracy value
Training: Batch 95, Epoch 0, 1 entries accuracy of OM core 0.9662 and TF 0.5775 
 Learned [1] of label(s) [0] , previously [47. 49.] of each.
 Learned [1] of label(s) [0] , previously [48. 49.] of each.
 Learned [1] of label(s) [0] , previously [49. 49.] of each.
 Learned [1] of label(s) [1] , previously [50. 49.] of each.
 Learned [1] of label(s) [1] , previously [50. 50.] of each.
800/800 [==============================] - 45s 57ms/step - loss: 0.6616 - accuracy: 0.5913
 (Number correct, number incorrect, number of tests): ( 772 28 800 ) %Correct:  96.5 %
False Positives by label: [11. 17.]
False Negatives by label: [17. 11.]
See

 Learned [1] of label(s) [0] , previously [82. 82.] of each.
 Learned [1] of label(s) [1] , previously [83. 82.] of each.
 Learned [1] of label(s) [1] , previously [83. 83.] of each.
 Learned [1] of label(s) [1] , previously [83. 84.] of each.
 Learned [1] of label(s) [0] , previously [83. 85.] of each.
 Learned [1] of label(s) [1] , previously [84. 85.] of each.
 Learned [1] of label(s) [0] , previously [84. 86.] of each.
800/800 [==============================] - 45s 57ms/step - loss: 0.5961 - accuracy: 0.7225
 (Number correct, number incorrect, number of tests): ( 774 26 800 ) %Correct:  96.75 %
False Positives by label: [ 7. 19.]
False Negatives by label: [19.  7.]
See   .last_score_accuracy for % accuracy value
Training: Batch 170, Epoch 0, 1 entries accuracy of OM core 0.9675 and TF 0.7225 
 Learned [1] of label(s) [0] , previously [85. 86.] of each.
 Learned [1] of label(s) [0] , previously [86. 86.] of each.
 Learned [1] of label(s) [0] , previously [87. 86.] of each.
 Learned 

 Learned [1] of label(s) [1] , previously [118. 117.] of each.
 Learned [1] of label(s) [0] , previously [118. 118.] of each.
 Learned [1] of label(s) [1] , previously [119. 118.] of each.
 Learned [1] of label(s) [0] , previously [119. 119.] of each.
 Learned [1] of label(s) [0] , previously [120. 119.] of each.
 Learned [1] of label(s) [1] , previously [121. 119.] of each.
800/800 [==============================] - 46s 57ms/step - loss: 0.5511 - accuracy: 0.7925
 (Number correct, number incorrect, number of tests): ( 773 27 800 ) %Correct:  96.625 %
False Positives by label: [ 9. 18.]
False Negatives by label: [18.  9.]
See   .last_score_accuracy for % accuracy value
Training: Batch 240, Epoch 0, 1 entries accuracy of OM core 0.9662 and TF 0.7925 
 Learned [1] of label(s) [1] , previously [121. 120.] of each.
 Learned [1] of label(s) [1] , previously [121. 121.] of each.
 Learned [1] of label(s) [1] , previously [121. 122.] of each.
 Learned [1] of label(s) [1] , previously [121. 123

 Learned [1] of label(s) [0] , previously [151. 154.] of each.
 Learned [1] of label(s) [0] , previously [152. 154.] of each.
 Learned [1] of label(s) [1] , previously [153. 154.] of each.
 Learned [1] of label(s) [0] , previously [153. 155.] of each.
 Learned [1] of label(s) [0] , previously [154. 155.] of each.
 Learned [1] of label(s) [0] , previously [155. 155.] of each.
800/800 [==============================] - 46s 57ms/step - loss: 0.5094 - accuracy: 0.8562
 (Number correct, number incorrect, number of tests): ( 773 27 800 ) %Correct:  96.625 %
False Positives by label: [ 8. 19.]
False Negatives by label: [19.  8.]
See   .last_score_accuracy for % accuracy value
Training: Batch 310, Epoch 0, 1 entries accuracy of OM core 0.9662 and TF 0.8562 
 Learned [1] of label(s) [0] , previously [156. 155.] of each.
 Learned [1] of label(s) [1] , previously [157. 155.] of each.
 Learned [1] of label(s) [0] , previously [157. 156.] of each.
 Learned [1] of label(s) [0] , previously [158. 156

 Learned [1] of label(s) [0] , previously [198. 189.] of each.
 Learned [1] of label(s) [1] , previously [199. 189.] of each.
 Learned [1] of label(s) [1] , previously [199. 190.] of each.
 Learned [1] of label(s) [1] , previously [199. 191.] of each.
800/800 [==============================] - 60s 74ms/step - loss: 0.4842 - accuracy: 0.8750
 (Number correct, number incorrect, number of tests): ( 773 27 800 ) %Correct:  96.625 %
False Positives by label: [ 8. 19.]
False Negatives by label: [19.  8.]
See   .last_score_accuracy for % accuracy value
Training: Batch 390, Epoch 0, 1 entries accuracy of OM core 0.9662 and TF 0.875 
 Learned [1] of label(s) [1] , previously [199. 192.] of each.
 Learned [1] of label(s) [1] , previously [199. 193.] of each.
 Learned [1] of label(s) [0] , previously [199. 194.] of each.
 Learned [1] of label(s) [0] , previously [200. 194.] of each.
 Learned [1] of label(s) [0] , previously [201. 194.] of each.
 Learned [1] of label(s) [0] , previously [202. 194.

 Learned [1] of label(s) [0] , previously [239. 230.] of each.
 Learned [1] of label(s) [1] , previously [240. 230.] of each.
800/800 [==============================] - 155s 194ms/step - loss: 0.4606 - accuracy: 0.9050
 (Number correct, number incorrect, number of tests): ( 772 28 800 ) %Correct:  96.5 %
False Positives by label: [ 9. 19.]
False Negatives by label: [19.  9.]
See   .last_score_accuracy for % accuracy value
Training: Batch 470, Epoch 0, 1 entries accuracy of OM core 0.965 and TF 0.905 
 Learned [1] of label(s) [1] , previously [240. 231.] of each.
 Learned [1] of label(s) [1] , previously [240. 232.] of each.
 Learned [1] of label(s) [0] , previously [240. 233.] of each.
 Learned [1] of label(s) [1] , previously [241. 233.] of each.
 Learned [1] of label(s) [0] , previously [241. 234.] of each.
 Learned [1] of label(s) [0] , previously [242. 234.] of each.
 Learned [1] of label(s) [0] , previously [243. 234.] of each.
 Learned [1] of label(s) [1] , previously [244. 234.]

800/800 [==============================] - 194s 243ms/step - loss: 0.4415 - accuracy: 0.9275
 (Number correct, number incorrect, number of tests): ( 773 27 800 ) %Correct:  96.625 %
False Positives by label: [ 8. 19.]
False Negatives by label: [19.  8.]
See   .last_score_accuracy for % accuracy value
Training: Batch 550, Epoch 0, 1 entries accuracy of OM core 0.9662 and TF 0.9275 
 Learned [1] of label(s) [0] , previously [275. 276.] of each.
 Learned [1] of label(s) [1] , previously [276. 276.] of each.
 Learned [1] of label(s) [1] , previously [276. 277.] of each.
 Learned [1] of label(s) [1] , previously [276. 278.] of each.
 Learned [1] of label(s) [1] , previously [276. 279.] of each.
 Learned [1] of label(s) [0] , previously [276. 280.] of each.
 Learned [1] of label(s) [0] , previously [277. 280.] of each.
 Learned [1] of label(s) [0] , previously [278. 280.] of each.
 Learned [1] of label(s) [1] , previously [279. 280.] of each.
 Learned [1] of label(s) [1] , previously [279. 2

In [ ]:
print(OM_learn_Vacc)

In [ ]:
# indicate when all animals are presented at least once
all_presented=1
while len(np.unique(labels_presented[0:all_presented])) != len(np.unique(labels_presented)):
    all_presented=all_presented+1
all_presented=np.where(np.array(graph_points) >= all_presented)[0][0]   #first index where all presented
print('first validation run after all animals were presented:',all_presented,'\nwhich begins with training instance',graph_points[all_presented])

In [ ]:
grange=int(graph_points[num_stored]*.02)     #90  #31 #90 #121  # choosing a nice zoom range
if grange==0:
    grange=num_stored
maxpoint=max(graph_points[0:grange])
print(maxpoint)

In [ ]:
grange

In [ ]:
# zoomed figure
print('first',graph_points[grange],'plots')

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)

plt.plot(graph_points[0:(all_presented+1)],OM_learn_Vacc[0:(all_presented+1)],color='darkblue')
plt.plot(graph_points[all_presented:grange],OM_learn_Vacc[all_presented:grange],color='cornflowerblue', label='OM Accuracy NEW')
plt.plot(graph_points[0:grange],tf_learn_Vacc[0:grange],color='orange', label='TF Accuracy NEW')



plt.legend(loc='lower right')
plt.ylabel('Validation Accuracy')
#plt.ylim([min(plt.ylim()),1])
plt.ylim([0.4,1.0])
plt.title("OM vs TF {} Accuracy within {} Batch={} Example(s) Trained".format(dataset_name,graph_points[grange],BATCH_SIZE))

plt.xlabel('Batch Number')
plt.show()
print('Labels  ',np.array(labels_presented)[0:grange].T)

In [ ]:
grange=np.int(num_stored*.05)

In [ ]:
print(max(graph_points),num_stored,graph_points[num_stored])

In [ ]:
#unzoomed figure
print('first',graph_points[grange],'plots')

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
#plt.plot(graph_points[0:grange],third_animal_OM_learn_Vacc[0:grange], label='OM Accuracy NEW')
#plt.plot(graph_points[0:grange],third_animal_tf_learn_Vacc[0:grange], label='TF Accuracy NEW')

plt.plot(graph_points[0:(all_presented+1)],OM_learn_Vacc[0:(all_presented+1)],color='darkblue')
plt.plot(graph_points[all_presented:grange],OM_learn_Vacc[all_presented:grange],color='cornflowerblue', label='OM')
plt.plot(graph_points[0:grange],tf_learn_Vacc[0:grange],color='orange', label='TF')



plt.legend(loc='lower right')
plt.ylabel('Validation Accuracy')
#plt.ylim([min(plt.ylim()),1])
plt.ylim([0.4,1.0])

plt.title("OM vs TF {} {} {} trained total".format(dataset_name,MODEL,graph_points[grange]))
#plt.xticks(np.arange(0,graph_points[num_stored]+1,int(graph_points[num_stored]/10)))  # Set label locations.
plt.xticks(np.arange(0,graph_points[grange]+1,int(graph_points[grange]/10)))  # Set label locations.



plt.xlabel('Batch Number')
plt.show()
print('Labels  ',np.array(labels_presented)[0:grange].T)